In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
from math import cos, sin
import numpy as np

L = np.array([100.0, 100.0])
T = 10.0
O, AB = np.array([40.0, 50.0]), np.array([7.0, 11.0])
WIDTH = np.array([5.0, 6.0])
(AMPLITUDE, GAMMA) = 1.0, 0.25

def condition(u: np.array, k: int) -> type(None):
    t = T / u.shape[0] * k
    xy = O + AB * np.array([cos(t), sin(t)])
    i, j = (L / u_k.shape[1:] * (np.outer(np.array([-1, 1]), WIDTH) + xy)).T
    u_k[i[0]:i[1], j[0]:j[1]] = AMPLITUDE

def calculate(u: np.ndarray) -> np.ndarray:
    for k in range(u.shape[0] - 1):
        A = u[k, 2:  , 1:-1]
        B = u[k,  :-2, 1:-1]
        C = u[k, 1:-1, 2:  ]
        D = u[k, 1:-1,  :-2]
        E = u[k, 1:-1, 1:-1]
        u[k + 1, 1:-1, 1:-1] = GAMMA * (A + B + C + D - 4.0 * E) + E
        condition(u[k + 1], k + 1)
    return u

In [ ]:
import matplotlib.pyplot as plt

def plot_heatmap(u_k: np.ndarray, k: int, vmin: float, vmax: float):
    print(F'\rk: {k}', flush=True, end='')
    plt.clf()

    plt.title(F"Temperature at t = {k} unit time")
    plt.xlabel("x")
    plt.ylabel("y")

    plt.pcolormesh(u_k, cmap=plt.cm.jet, vmin=vmin, vmax=vmax)
    plt.colorbar()

    return plt

In [ ]:
from matplotlib.animation import FuncAnimation

u = np.zeros((160, 384, 384))
condition(u, 0)
u = calculate(u, GAMMA)
vmin, vmax = u.min(), u.max()
animate = lambda k: plot_heatmap(u[k], k, vmin, vmax)
animation = FuncAnimation(plt.figure(), animate, interval=1, frames=len(u), repeat=False)
animation.save("heat_equation_solution.gif")

In [ ]:
from timeit import default_timer as timer

from pydmd import DMD

now = timer()
dmd = DMD(svd_rank=2, exact=False).fit(u.reshape(u.shape[0], -1).T)
print(timer() - now)

In [ ]:
# np.all(u.reshape(u.shape[0], -1)[0] == u[0].reshape(-1))
u.reshape(u.shape[0], -1).T.shape

In [ ]:
dmd.amplitudes

In [ ]:
dmd.dynamics

In [ ]:
data = dmd.reconstructed_data
data[(np.abs(data) > 1E-4)]

In [ ]:
u.reshape(u.shape[0], -1)